In [1]:
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
import awkward as ak
import matplotlib.pyplot as plt
import numba
from numba import njit
import numpy as np
from pocket_coffea.lib.parton_provenance import reverse_index_array
from pocket_coffea.lib.deltaR_matching import object_matching


In [201]:
j=0
filename ="/pnfs/psi.ch/cms/trivcat/store/mc/Run3Summer22EENanoAODv12/VBFHHto4B_CV_1_C2V_1_C3_1_TuneCP5_13p6TeV_madgraph-pythia8/NANOAODSIM/130X_mcRun3_2022_realistic_postEE_v6-v3/80000/508f1313-c598-48b5-984e-86cd0f02a820.root"
events = NanoEventsFactory.from_root(filename, schemaclass=NanoAODSchema, entry_start=j, entry_stop=j+10).events()
print("Events read:", len(events))

Events read: 10


In [202]:
jet=ak.with_field(
    events.Jet, ak.local_index(events.Jet, axis=1), "index"
)

In [203]:
# add column with the index
events.GenPart = ak.with_field(
    events.GenPart, ak.local_index(events.GenPart, axis=1), "index"
)
print(events.GenPart.index[0])
print(events.GenPart.index[1])
# get last bquark copy
isB = abs(events.GenPart.pdgId) == 5
isLast = events.GenPart.hasFlags(["isLastCopy"])
isHard = events.GenPart.hasFlags(["fromHardProcess"])
bquarks_last = events.GenPart[isB & isLast & isHard]

print(bquarks_last.pdgId, bquarks_last.genPartIdxMother, bquarks_last.childrenIdxG, bquarks_last.hasFlags(["isLastCopy"]))
bquark_last_children=bquarks_last.children
print(bquark_last_children.pdgId, bquark_last_children.genPartIdxMother, bquark_last_children.hasFlags(["isLastCopy"]))

b_quarks = bquarks_last
while True:
    print("\nloop")
    b_mother = events.GenPart[b_quarks.genPartIdxMother]
    mask_mother = (abs(b_mother.pdgId) == 5) | ((b_mother.pdgId) == 25)

    bquarks_last = bquarks_last[mask_mother]
    b_quarks = b_quarks[mask_mother]
    b_mother = b_mother[mask_mother]
    print(
        "old: ",
        "pdg",
        b_quarks.pdgId,
        "mother_idx",
        b_quarks.genPartIdxMother,
        "pt",
        b_quarks.pt,
        "index",
        b_quarks.index,
        "fromHardProcess",
        b_quarks.hasFlags(["fromHardProcess"]),
    )
    print(
        "mother: ",
        "pdg",
        b_mother.pdgId,
        "mother_idx",
        b_mother.genPartIdxMother,
        "pt",
        b_mother.pt,
    )
    # break
    b_quarks = ak.where(abs(b_mother.pdgId) == 5, b_mother, b_quarks)
    print(
        "new: ",
        "pdg",
        b_quarks.pdgId,
        "mother_idx",
        b_quarks.genPartIdxMother,
        "pt",
        b_quarks.pt,
    )

    print(ak.any(abs(b_mother.pdgId) != 25, axis=1))
    if ak.all(abs(b_mother.pdgId) == 25):
        break

print(b_quarks.pdgId, b_quarks.genPartIdxMother, b_quarks.pt)
print(b_quarks.genPartIdxMother, len(b_quarks.genPartIdxMother))
print(b_quarks.index, len(b_quarks.index))
print(bquarks_last.pdgId, bquarks_last.index,  bquarks_last.pt, len(bquarks_last.index), bquarks_last.hasFlags(["fromHardProcess"]))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, ... 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, ... 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46]
[[5, -5, 5, -5], [5, 5, -5, -5], [5, -5, ... 5, -5], [5, -5, 5, -5], [5, -5, 5, -5]] [[12, 13, 15, 16], [14, 16, 15, 17], [18, ... [14, 16, 12, 13], [14, 15, 16, 17]] [[[], [], [], []], [[], [], [], []], [[], ... [[], [], [], []], [[], [], [], []]] [[True, True, True, True], [True, True, ... True, True], [True, True, True, True]]
[[[], [], [], []], [[], [], [], []], [[], ... [[], [], [], []], [[], [], [], []]] [[[], [], [], []], [[], [], [], []], [[], ... [[], [], [], []], [[], [], [], []]] [[[], [], [], []], [[], [], [], []], [[], ... [[], [], [], []], [[], [], [], []]]

loop
old:  pdg [[5, -5, 5, -5], [5, 5, -5, -5], [5, -5, ... 5, -5], [5, -5, 5, -5], [5, -5, 5, -5]] mother_idx [[12, 13, 15, 16], [14, 16, 15, 17], [18, ... [14, 16, 12, 13], [14, 15, 16, 17]] pt [[16.9, 132, 55, 39.6], [51.2, 101, 102, ... 12.6, 103, 13.

In [204]:
matched_bquarks_higgs, matched_jets_higgs, deltaR_matched_higgs = (
    object_matching(
        bquarks_last,
        jet,
        dr_min=0.4,
    )
)

In [205]:
print(matched_bquarks_higgs.index, matched_bquarks_higgs.pt, matched_bquarks_higgs.pdgId)
print(matched_jets_higgs.index, matched_jets_higgs.pt)
print(deltaR_matched_higgs)

[[None, None, 20, None, 21, None], [20, ... [24, None, 31, None, 29, None, None]] [[None, None, 55, None, 39.6, None], ... [261, None, 217, None, 65, None, None]] [[None, None, 5, None, -5, None], [5, -5, ... [5, None, 5, None, -5, None, None]]
[[None, None, 2, None, 4, None], [0, 1, ... None], [0, None, 2, None, 4, None, None]] [[None, None, 37.3, None, 28.4, None], ... 270, None, 193, None, 62.8, None, None]]
[[None, None, 0.0962, None, 0.0598, None, ... 0.00319, None, 0.0236, None, None]]


In [206]:
print(matched_jets_higgs.index)
print(~ak.is_none(matched_jets_higgs.index, axis=1))


jets_no_higgs = jet[ak.is_none(matched_jets_higgs.index, axis=1)]
jets_no_higgs=ak.pad_none(jets_no_higgs, 2, axis=1)
print(jets_no_higgs.index, jets_no_higgs.pt)


[[None, None, 2, None, 4, None], [0, 1, ... None], [0, None, 2, None, 4, None, None]]
[[False, False, True, False, True, False, ... True, False, True, False, False]]
[[0, 1, 3, 5], [3, 4], [1, 4], [0, 2], ... 5, 6, 7], [0, 2, 5, 6, 7], [1, 3, 5, 6]] [[73.5, 66.4, 28.5, 15.5], [41.2, 19.8], ... 17.5, 17.1], [210, 138, 49.2, 26.4]]


# get the jets not from higgs using numba

In [207]:
@njit
def get_jets_no_higgs(jets_index_all, jets_from_higgs_idx):
    # jets_no_higgs_idx = np.zeros(jet_offset_no_higgs[-1], dtype="int64")-1
    jets_no_higgs_idx = jets_index_all
    print(len(jets_no_higgs_idx), jets_no_higgs_idx)

    # if jets_from_higgs_idx put -1
    for ijet_higgs in jets_from_higgs_idx:
        jets_no_higgs_idx[ijet_higgs] = -1

    return jets_no_higgs_idx

In [208]:
num=2
matched_jet_higgs_idx_not_none = matched_jets_higgs.index[~ak.is_none(matched_jets_higgs.index, axis=1)][:,:num]

jet_flat=ak.flatten(jet)
jet_offsets = np.concatenate([[0],np.cumsum(ak.to_numpy(ak.num(jet, axis=1), allow_missing=True))])
jet_offset_no_higgs = jet_offsets-num
num_jets=ak.num(jet, axis=1)

local_index_all = ak.local_index(jet, axis=1)
jets_index_all = ak.to_numpy(ak.flatten(local_index_all+jet_offsets[:-1]), allow_missing=True)
jets_from_higgs_idx=ak.to_numpy(ak.flatten(matched_jet_higgs_idx_not_none+jet_offsets[:-1]), allow_missing=False)

nevents=len(jet)

print(matched_jet_higgs_idx_not_none)
print(jet_offsets)
print(jet_offset_no_higgs)
print(local_index_all)
print(jets_index_all)
print(jets_from_higgs_idx)

jets_no_higgs_idx = get_jets_no_higgs(jets_index_all, jets_from_higgs_idx)
print(jets_no_higgs_idx)
print(jet_offsets[1:])
jets_no_higgs_idx_unflat = ak.unflatten(jets_no_higgs_idx, num_jets)-jet_offsets[:-1]
print(jets_no_higgs_idx_unflat)

jet_no_higgs_numba=jet[jets_no_higgs_idx_unflat>=0]
print(jet_no_higgs_numba.index, jet_no_higgs_numba.pt)

[[2, 4], [0, 1], [0, 2], [1, 3], [0, 3], [0, 1], [0, 2], [0, 1], [1, 3], [0, 2]]
[ 0  6 11 16 21 27 33 36 44 52 59]
[-2  4  9 14 19 25 31 34 42 50 57]
[[0, 1, 2, 3, 4, 5], [0, 1, 2, 3, 4], ... 3, 4, 5, 6, 7], [0, 1, 2, 3, 4, 5, 6]]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58]
[ 2  4  6  7 11 13 17 19 21 24 27 28 33 35 36 37 45 47 52 54]
59 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58]
[ 0  1 -1  3 -1  5 -1 -1  8  9 10 -1 12 -1 14 15 16 -1 18 -1 20 -1 22 23
 -1 25 26 -1 -1 29 30 31 32 -1 34 -1 -1 -1 38 39 40 41 42 43 44 -1 46 -1
 48 49 50 51 -1 53 -1 55 56 57 58]
[ 6 11 16 21 27 33 36 44 52 59]
[[0, 1, -1, 3, -1, 5], [-7, -7, 2, 3, 4], ... 4, 5, 6, 7], [-53, 1, -53, 3, 4, 5, 6]]
[[0, 1, 3, 5], [2, 3, 4], [1, 3, 4], [0, .

In [209]:
# raise SystemExit
# matched_jet_higgs_idx_not_none = matched_jets_higgs.index[~ak.is_none(matched_jets_higgs.index, axis=1)]
# print(matched_jet_higgs_idx_not_none[:,0], type(matched_jet_higgs_idx_not_none[:,0]))
# print( len(jet), ak.local_index(jet, axis=0))
# a=np.array([[0,1],[1,0], [0,1], [1,0], [0,1], [0,1], [1,0], [0,1], [1,0], [0,1]])
# print(type(a), a)
# b=ak.to_numpy( matched_jet_higgs_idx_not_none[:,:2])
# print(b, type(b))
# # b=( matched_jet_higgs_idx_not_none[:,0])
# idx_none=ak.where(ak.ones_like(jet.pt, dtype=np.int32)==1, None, None)
# print(idx_none)
# #where the index matched_jet_higgs_idx_not_none put the index of the jet otherwise put None
# print(idx_none[matched_jet_higgs_idx_not_none])

# #set
# jets_from_higgs_to_none = jet
# jets_from_higgs_to_none[matched_jet_higgs_idx_not_none] = None


# # jets_no_higgs=jet[b]
# print(jets_no_higgs.index, jets_no_higgs.pt)

choose vbf jets as the two jets with the highest pt that are not from higgs decay

In [210]:
vbf_jets_pt =jets_no_higgs[:,:2]
print(vbf_jets_pt.index, vbf_jets_pt.pt)

[[0, 1], [3, 4], [1, 4], [0, 2], [1, 2, ... 3, 4], [1, None], [2, 3], [0, 2], [1, 3]] [[73.5, 66.4], [41.2, 19.8], [250, 39.8], ... [60.7, 48.1], [114, 67.8], [210, 138]]


choose higgs jets as the two jets with the highest mjj that are not from higgs decay

In [211]:
jet_combinations_idx = ak.argcombinations(jets_no_higgs, 2)
print(jet_combinations_idx)
jet_combinations=ak.combinations(jets_no_higgs, 2)
print(jet_combinations)
print(jet_combinations[0][0])
print(jet_combinations[0,0]["0"].index, jet_combinations[0,0]["1"].index)

[[(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, ... ), (0, 3), (1, 2), (1, 3), (2, 3)]]
[[(Jet, Jet), (Jet, Jet), (Jet, Jet), (Jet, ... Jet, Jet), (Jet, Jet), (Jet, Jet)]]
(Jet, Jet)
0 1


In [212]:
jet_combinations_mass = (jet_combinations["0"] + jet_combinations["1"]).mass
print(jet_combinations_mass)

[[681, 228, 1.63e+03, 78.8, 168, 212], ... [987, 1.34e+03, 82.2, 162, 561, 758]]


In [213]:
jet_combinations_mass_ordered = jet_combinations_mass[ak.argsort(jet_combinations_mass, axis=1, ascending=False)]
print(jet_combinations_mass_ordered)
jet_combinations_mass_ordered_idx = ak.argsort(jet_combinations_mass, axis=1, ascending=False)
print(jet_combinations_mass_ordered_idx)

[[1.63e+03, 681, 228, 212, 168, 78.8], ... [1.34e+03, 987, 758, 561, 162, 82.2]]
[[2, 0, 1, 5, 4, 3], [0], [0], [0], ... 4, 2, 5, 9, 7, 3, 1, 8], [1, 0, 5, 4, 3, 2]]


In [214]:
jet_combinations_mass_max_idx = ak.to_numpy(jet_combinations_mass_ordered_idx[:,0])
print(jet_combinations_mass_max_idx)
print(jet_combinations)
print(ak.local_index(jet_combinations, axis=0))
jets_max_mass=jet_combinations[ak.local_index(jet_combinations, axis=0), jet_combinations_mass_max_idx]
print(jets_max_mass["0"].index, jets_max_mass["1"].index)
print(jets_max_mass["0"].pt, jets_max_mass["1"].pt)

[2 0 0 0 0 0 0 0 0 1]
[[(Jet, Jet), (Jet, Jet), (Jet, Jet), (Jet, ... Jet, Jet), (Jet, Jet), (Jet, Jet)]]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 3, 1, 0, 1, 3, 1, 2, 0, 1] [5, 4, 4, 2, 2, 4, None, 3, 2, 5]
[73.5, 41.2, 250, 72.4, 95.2, 48.5, 41.8, 60.7, 114, 210] [15.5, 19.8, 39.8, 39.5, 61.3, 20.9, None, 48.1, 67.8, 49.2]


In [220]:
print(jets_no_higgs.index)
print(jet_combinations["0"].index)
print(jet_combinations["1"].index)
print(jet_combinations_mass_max_idx)
print(jets_max_mass["0"].index)
print(jets_max_mass["1"].index)

[[0, 1, 3, 5], [3, 4], [1, 4], [0, 2], ... 5, 6, 7], [0, 2, 5, 6, 7], [1, 3, 5, 6]]
[[0, 0, 0, 1, 1, 3], [3], [1], [0], ... 0, 0, 2, 2, 2, 5, 5, 6], [1, 1, 1, 3, 3, 5]]
[[1, 3, 5, 3, 5, 5], [4], [4], [2], ... 6, 7, 5, 6, 7, 6, 7, 7], [3, 5, 6, 5, 6, 6]]
[2 0 0 0 0 0 0 0 0 1]
[0, 3, 1, 0, 1, 3, 1, 2, 0, 1]
[5, 4, 4, 2, 2, 4, None, 3, 2, 5]


In [224]:
idx_0=ak.to_numpy(jets_max_mass["0"].index)
idx_1=ak.to_numpy(jets_max_mass["1"].index)
print(idx_0, idx_1)
print(jets_max_mass["1"].index)
vbf_jets_mass_max_0 = ak.unflatten(jet[ak.local_index(jets_no_higgs, axis=0), idx_0], ak.ones_like(jets_max_mass["0"].index))
vbf_jets_mass_max_1 = ak.unflatten(jet[ak.local_index(jets_no_higgs, axis=0), idx_1], ak.ones_like(jets_max_mass["1"].index))
print(vbf_jets_mass_max_0.index, vbf_jets_mass_max_0.pt, vbf_jets_mass_max_0)
print(vbf_jets_mass_max_1.index, vbf_jets_mass_max_1.pt, vbf_jets_mass_max_1)
vbf_jets_mass_max = ak.concatenate([vbf_jets_mass_max_0, vbf_jets_mass_max_1], axis=1)
print(vbf_jets_mass_max.index, vbf_jets_mass_max.pt)

[0 3 1 0 1 3 1 2 0 1] [5 4 4 2 2 4 -- 3 2 5]
[5, 4, 4, 2, 2, 4, None, 3, 2, 5]


ValueError: cannot mix missing values in slice with NumPy-style advanced indexing

(https://github.com/scikit-hep/awkward-1.0/blob/1.10.3/src/libawkward/Content.cpp#L1662)

In [ ]:
a=ak.Array([1,2,3])
b=ak.Array([4,5,6])
a_unflatten=ak.singletons(a)
b_unflatten=ak.singletons(b)
c_list=[a_unflatten, b_unflatten]
c_concat=ak.concatenate(c_list, axis=1)
print(c_concat)

[[1, 4], [2, 5], [3, 6]]
